In [172]:
!pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [173]:
!pip install pyECLAT

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [174]:
!pip install pyfpgrowth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [175]:
# Import headers
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

In [176]:
#Data preprocessing
dataset = pd.read_csv('/content/groceries - groceries.csv', header = 0)
Data=dataset
rows = len(Data.axes[0])
cols = len(Data.axes[1])
Data.columns = range(Data.shape[1])
rows

9834

In [177]:
# Intializing the list
transacts = []
# populating a list of transactions
for i in range(0, rows): 
  transacts.append([str(Data.values[i,j]) for j in range(0, cols)])

In [178]:
#Apriori algorithm
from apyori import apriori
ap = apriori(transactions = transacts, min_support = 0.2, use_colnames=True) # generate rules
# list of results coming from the apriori model
rslt = list(ap)

rslt

[RelationRecord(items=frozenset({'nan'}), support=0.9998983119788489, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'nan'}), confidence=0.9998983119788489, lift=1.0)]),
 RelationRecord(items=frozenset({'whole milk'}), support=0.2555419971527354, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'whole milk'}), confidence=0.2555419971527354, lift=1.0)]),
 RelationRecord(items=frozenset({'whole milk', 'nan'}), support=0.2554403091315843, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'nan', 'whole milk'}), confidence=0.2554403091315843, lift=1.0), OrderedStatistic(items_base=frozenset({'nan'}), items_add=frozenset({'whole milk'}), confidence=0.2554662869927794, lift=0.9997037271337018), OrderedStatistic(items_base=frozenset({'whole milk'}), items_add=frozenset({'nan'}), confidence=0.9996020692399522, lift=0.9997037271337018)])]

In [194]:
%timeit -n 100 -r 10 ap

100 loops, best of 10: 54.3 ns per loop


In [180]:
#ECLAT algorithm
from pyECLAT import ECLAT

# loading transactions DataFrame to ECLAT class
eclat = ECLAT(data=Data)

# DataFrame of binary values
eclat.df_bin


,grapes,specialty cheese,jam,fish,frozen meals,herbs,frozen dessert,hard cheese,make up remover,candy,...,canned fruit,white bread,bags,pet care,chocolate marshmallow,soft cheese,meat spreads,dishes,red/blush wine,ice cream
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9829,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9830,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9831,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9832,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [181]:
items_total = eclat.df_bin.astype(int).sum(axis=0)

items_total

grapes              220
specialty cheese     84
jam                  53
fish                 29
frozen meals        279
                   ... 
soft cheese         168
meat spreads         42
dishes              173
red/blush wine      189
ice cream           246
Length: 169, dtype: int64

In [182]:
# count items in each row
items_per_transaction = eclat.df_bin.astype(int).sum(axis=1)

items_per_transaction

0        3
1        1
2        4
3        4
4        5
        ..
9829    17
9830     1
9831    10
9832     4
9833     5
Length: 9834, dtype: int64

In [183]:
df = pd.DataFrame({'items': items_total.index, 'transactions': items_total.values}) 

# cloning pandas DataFrame for visualization purpose  
df_table = df.sort_values("transactions", ascending=False)

#  Top 5 most popular products/items
df_table.head(5)

,items,transactions
106,whole milk,2513
60,other vegetables,1903
131,rolls/buns,1809
35,soda,1715
79,yogurt,1372


In [184]:
eclat_indices, eclat_supports = eclat.fit(min_support=0.2)

Combination 1 by 1


1it [00:00, 35.29it/s]


Combination 2 by 2


0it [00:00, ?it/s]


Combination 3 by 3


0it [00:00, ?it/s]


In [185]:
len(eclat_supports)

1

In [186]:
print(eclat_supports)

{'whole milk': 0.2555419971527354}


In [196]:
%timeit -n 100 -r 10 eclat_supports

100 loops, best of 10: 41.5 ns per loop


In [188]:
import pyfpgrowth

In [189]:
min_support= 0.2*rows
patterns = pyfpgrowth.find_frequent_patterns(transacts, min_support)
patterns

{('nan',): 271325,
 ('nan', 'nan'): 3670683,
 ('nan', 'whole milk'): 63422,
 ('whole milk',): 2513}

In [190]:
%timeit -n 100 -r 10 patterns

100 loops, best of 10: 44 ns per loop
